In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import torch 
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
   device = torch.device("cpu")


%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM,Dropout,BatchNormalization,Bidirectional,Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount("/content/drive/")
LIBRARY_PATH = '/content/drive/MyDrive/NLP PROJECT/'


Tensorflow version 2.8.0
Mounted at /content/drive/


### Understanding data

In [ ]:
train = pd.read_csv(LIBRARY_PATH + "preprocessed_data/clean_train.csv")
test = pd.read_csv(LIBRARY_PATH + "preprocessed_data/clean_test.csv")

In [ ]:
train.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,keyword,location,text,target,hashtags,text without sw,text with sw,text without sw_freq
0,0,0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,['earthquake'],deeds reason earthquake may allah forgive us,our deeds are the reason of this earthquake ma...,deeds reason earthquake may allah forgive us
1,1,1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,[],forest fire near la ronge sask canada,forest fire near la ronge sask canada,forest fire near la ronge sask canada
2,2,2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,[],residents asked shelter place notified officer...,all residents asked to shelter in place are ...,residents asked shelter place notified officer...
3,3,3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,['wildfires'],people receive wildfires evacuation orders cal...,people receive wildfires evacuation orders i...,people receive wildfires evacuation orders cal...
4,4,4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"['Alaska', 'wildfires']",got sent photo ruby alaska smoke wildfires pou...,just got sent this photo from ruby alaska as s...,got sent photo ruby alaska smoke wildfires pou...


## BERT on raw dataset

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 596 kB 60.9 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 342 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 55.2 MB/s 
     |████████████████████████████████| 1.1 MB 65.1 MB/s 
     |████████████████████████████████| 136 kB 73.3 MB/s 
     |████████████████████████████████| 127 kB 76.9 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
     |████████████████████████████████| 271 kB 67.7 MB/s 
     |████████████████████████████████| 144 kB 77.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ER

In [ ]:
from datasets import load_dataset
from transformers import pipeline
from transformers import TrainingArguments, Trainer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import numpy as np
import pandas as pd
from datasets import Dataset, load_metric
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

# load tokenizer and model
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler

In [ ]:
train_data = pd.read_csv(LIBRARY_PATH + "twitter_dataset/train.csv")
train_data = train_data[['text', 'target']]
train_data.rename(columns={"target": "label"}, inplace=True)

test = pd.read_csv(LIBRARY_PATH + "twitter_dataset/test.csv")
test = test[['text']]
test.rename(columns={"target": "label"}, inplace=True)

## splitting train and validation data
train, valid = train_test_split(train_data, test_size=0.1, shuffle=True, stratify=train_data['label'])

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# define parameters 
batch_size = 16
lr = 5e-5
num_epochs = 3
metric_val = "f1"

# load model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model.to(device)

# tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', \
                    truncation = True)

train_data = Dataset.from_pandas(train)
dev_data = Dataset.from_pandas(valid)
test_data = Dataset.from_pandas(test)

encoded_dataset_train = train_data.map(tokenize_function, batched=True)
encoded_dataset_dev = dev_data.map(tokenize_function, batched=True)
encoded_dataset_test = test_data.map(tokenize_function, batched=True)


# tokenized_datasets = train.map(tokenize_function)
encoded_dataset_train = encoded_dataset_train.remove_columns(["text", "__index_level_0__"])
encoded_dataset_train = encoded_dataset_train.rename_column("label", "labels")
encoded_dataset_train.set_format("torch")

encoded_dataset_dev = encoded_dataset_dev.remove_columns(["text", "__index_level_0__"])
encoded_dataset_dev = encoded_dataset_dev.rename_column("label", "labels")
encoded_dataset_dev.set_format("torch")

# dataloader = data_utils.DataLoader(dataset, batch_size=N)

# encoded_dataset_train = encoded_dataset_train.batch(batch_size=batch_size)
# encoded_dataset_dev = encoded_dataset_dev.batch(batch_size=batch_size)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
# encoded_dataset_train

In [ ]:
# train_data

In [ ]:
# train

In [ ]:
# train_dataloader = DataLoader(encoded_dataset_train, shuffle=True, batch_size=batch_size)
# eval_dataloader = DataLoader(encoded_dataset_dev, batch_size=batch_size)

In [ ]:
# train_dataloader

In [ ]:
train_dataloader = DataLoader(encoded_dataset_train, shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(encoded_dataset_dev, batch_size=batch_size)

# optimizer and learning rate schedule
optimizer = AdamW(model.parameters(), lr=lr)

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, \
                             num_warmup_steps=0, \
                             num_training_steps=num_training_steps)


from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

metric = load_metric(metric_val)
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

## 15 min 78.9% F1 score


  0%|          | 0/1287 [00:00<?, ?it/s]

{'f1': 0.7898894154818326}

In [ ]:
model=model.to(torch.device('cpu'))
model.eval()

test_set = list(test['text'])
test_predictions = []

# inference_set =  ['I was on hold for more than 60 minutes, their customer support service is a nightmare',  \
#                  'I really like the new design of your website', \
#                  'The mobile application is glitchy and not user friendly', \
#                  'The product does wonders for your diet']

for txt in test_set:
  encoded_input = tokenizer(txt, padding="max_length", \
                     truncation=True, return_tensors='pt')
  output = model(**encoded_input)
  logits = output.logits
  prediction = torch.argmax(logits, dim=-1)
  test_predictions.append(prediction)


## BERT on preprocessed data

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from datasets import load_dataset
from transformers import pipeline
from transformers import TrainingArguments, Trainer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import numpy as np
import pandas as pd
from datasets import Dataset, load_metric
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

# load tokenizer and model
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler

In [ ]:
train = pd.read_csv(LIBRARY_PATH + "preprocessed_data/clean_train.csv")
test = pd.read_csv(LIBRARY_PATH + "preprocessed_data/clean_test.csv")

# train_data = pd.read_csv(LIBRARY_PATH + "twitter_dataset/train.csv")
train_data = train[['text with sw', 'target']]
train_data.rename(columns={"target": "label"}, inplace=True)
train_data.rename(columns={"text with sw": "text"}, inplace=True)

# test = pd.read_csv(LIBRARY_PATH + "twitter_dataset/test.csv")
test = test[['text with sw']]
test.rename(columns={"text with sw": "text"}, inplace=True)
# test.rename(columns={"target": "label"}, inplace=True)

## splitting train and validation data
train, valid = train_test_split(train_data, test_size=0.1, shuffle=True, stratify=train_data['label'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# define parameters 
batch_size = 16
lr = 5e-5
num_epochs = 3
metric_val = "f1"

# load model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model.to(device)

# tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', \
                    truncation = True)


train_data = Dataset.from_pandas(train)
dev_data = Dataset.from_pandas(valid)
test_data = Dataset.from_pandas(test)

encoded_dataset_train = train_data.map(tokenize_function, batched=True)
encoded_dataset_dev = dev_data.map(tokenize_function, batched=True)
encoded_dataset_test = test_data.map(tokenize_function, batched=True)

encoded_dataset_train = encoded_dataset_train.remove_columns([ "text", "__index_level_0__"])
encoded_dataset_train = encoded_dataset_train.rename_column("label", "labels")
encoded_dataset_train.set_format("torch")

encoded_dataset_dev = encoded_dataset_dev.remove_columns([ "text", "__index_level_0__"])
encoded_dataset_dev = encoded_dataset_dev.rename_column("label", "labels")
encoded_dataset_dev.set_format("torch")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
train_dataloader = DataLoader(encoded_dataset_train, shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(encoded_dataset_dev, batch_size=batch_size)

# optimizer and learning rate schedule
optimizer = AdamW(model.parameters(), lr=lr)

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, \
                             num_warmup_steps=0, \
                             num_training_steps=num_training_steps)


from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

metric = load_metric(metric_val)
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()
## 79.5% 15

  0%|          | 0/1287 [00:00<?, ?it/s]

{'f1': 0.7957317073170731}

## RoBERTa on raw dataset

In [ ]:
from datasets import load_dataset
from transformers import pipeline
from transformers import TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np
import pandas as pd
from datasets import Dataset, load_metric
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

# load tokenizer and model
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler

In [ ]:
train_data = pd.read_csv(LIBRARY_PATH + "twitter_dataset/train.csv")
train_data = train_data[['text', 'target']]
train_data.rename(columns={"target": "label"}, inplace=True)

test = pd.read_csv(LIBRARY_PATH + "twitter_dataset/test.csv")
test = test[['text']]
test.rename(columns={"target": "label"}, inplace=True)

## splitting train and validation data
train, valid = train_test_split(train_data, test_size=0.1, shuffle=True, stratify=train_data['label'])

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# define parameters 
batch_size = 16
lr = 5e-5
num_epochs = 3
metric_val = "f1"

# load model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model.to(device)

# tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', \
                    truncation = True)

train_data = Dataset.from_pandas(train)
dev_data = Dataset.from_pandas(valid)
test_data = Dataset.from_pandas(test)

encoded_dataset_train = train_data.map(tokenize_function, batched=True)
encoded_dataset_dev = dev_data.map(tokenize_function, batched=True)
encoded_dataset_test = test_data.map(tokenize_function, batched=True)


# tokenized_datasets = train.map(tokenize_function)
encoded_dataset_train = encoded_dataset_train.remove_columns(["text", "__index_level_0__"])
encoded_dataset_train = encoded_dataset_train.rename_column("label", "labels")
encoded_dataset_train.set_format("torch")

encoded_dataset_dev = encoded_dataset_dev.remove_columns(["text", "__index_level_0__"])
encoded_dataset_dev = encoded_dataset_dev.rename_column("label", "labels")
encoded_dataset_dev.set_format("torch")

# dataloader = data_utils.DataLoader(dataset, batch_size=N)

# encoded_dataset_train = encoded_dataset_train.batch(batch_size=batch_size)
# encoded_dataset_dev = encoded_dataset_dev.batch(batch_size=batch_size)


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
train_dataloader = DataLoader(encoded_dataset_train, shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(encoded_dataset_dev, batch_size=batch_size)

# optimizer and learning rate schedule
optimizer = AdamW(model.parameters(), lr=lr)

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, \
                             num_warmup_steps=0, \
                             num_training_steps=num_training_steps)


from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

metric = load_metric(metric_val)
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

# 79.8 - 32 min

  0%|          | 0/1287 [00:00<?, ?it/s]

{'f1': 0.7981510015408321}

## RoBERTa on preprocessed dataset

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.2 MB 9.8 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
     |████████████████████████████████| 6.6 MB 48.0 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 342 kB 7.2 MB/s 
     |████████████████████████████████| 1.1 MB 17.2 MB/s 
     |████████████████████████████████| 212 kB 35.8 MB/s 
     |████████████████████████████████| 136 kB 41.6 MB/s 
     |████████████████████████████████| 127 kB 42.1 MB/s 
     |████████████████████████████████| 144 kB 34.7 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 45.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ER

In [ ]:
from datasets import load_dataset
from transformers import pipeline
from transformers import TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np
import pandas as pd
from datasets import Dataset, load_metric
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

# load tokenizer and model
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler

In [ ]:
train = pd.read_csv(LIBRARY_PATH + "preprocessed_data/clean_train.csv")
test = pd.read_csv(LIBRARY_PATH + "preprocessed_data/clean_test.csv")

train_data = train[['text with sw', 'target']]
train_data.rename(columns={"target": "label"}, inplace=True)
train_data.rename(columns={"text with sw": "text"}, inplace=True)

test = test[['text with sw']]
test.rename(columns={"text with sw": "text"}, inplace=True)

## splitting train and validation data
train, valid = train_test_split(train_data, test_size=0.1, shuffle=True, stratify=train_data['label'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# define parameters 
# tried parameters
# Batch size: 16, 32
# Learning rate (Adam): 5e-5, 3e-5, 2e-5
# Number of epochs: 2, 3, 4


batch_size = 16
lr = 5e-5
num_epochs = 3
metric_val = "f1"

# load model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model.to(device)

# tokenize dataset  
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', \
                    truncation = True)

train_data = Dataset.from_pandas(train)
dev_data = Dataset.from_pandas(valid)
test_data = Dataset.from_pandas(test)

encoded_dataset_train = train_data.map(tokenize_function, batched=True)
encoded_dataset_dev = dev_data.map(tokenize_function, batched=True)
encoded_dataset_test = test_data.map(tokenize_function, batched=True)


# tokenized_datasets = train.map(tokenize_function)
encoded_dataset_train = encoded_dataset_train.remove_columns(["text", "__index_level_0__"])
encoded_dataset_train = encoded_dataset_train.rename_column("label", "labels")
encoded_dataset_train.set_format("torch")

encoded_dataset_dev = encoded_dataset_dev.remove_columns(["text", "__index_level_0__"])
encoded_dataset_dev = encoded_dataset_dev.rename_column("label", "labels")
encoded_dataset_dev.set_format("torch")


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
train_dataloader = DataLoader(encoded_dataset_train, shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(encoded_dataset_dev, batch_size=batch_size)

# optimizer and learning rate schedule
optimizer = AdamW(model.parameters(), lr=lr)

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, \
                             num_warmup_steps=0, \
                             num_training_steps=num_training_steps)


from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

metric = load_metric(metric_val)
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

# 80.7 

  0%|          | 0/1287 [00:00<?, ?it/s]

{'f1': 0.8073394495412844}

## RoBERTa with data preprocessing (with hyper=parameter tuning)

In [ ]:
from datasets import load_dataset
from transformers import pipeline
from transformers import TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np
import pandas as pd
from datasets import Dataset, load_metric
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

# load tokenizer and model
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler

In [ ]:
train = pd.read_csv(LIBRARY_PATH + "preprocessed_data/clean_train.csv")
test = pd.read_csv(LIBRARY_PATH + "preprocessed_data/clean_test.csv")

train_data = train[['text with sw', 'target']]
train_data.rename(columns={"target": "label"}, inplace=True)
train_data.rename(columns={"text with sw": "text"}, inplace=True)

test = test[['text with sw']]
test.rename(columns={"text with sw": "text"}, inplace=True)

## splitting train and validation data
train, valid = train_test_split(train_data, test_size=0.1, shuffle=True, stratify=train_data['label'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# define parameters 
# tried parameters
# Batch size: 16, 32
# Learning rate (Adam): 5e-5, 3e-5, 2e-5
# Number of epochs: 2, 3, 4


batch_size = 16
lr = 5e-5
num_epochs = 3
metric_val = "f1"

# load model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model.to(device)

# tokenize dataset  
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', \
                    truncation = True)

train_data = Dataset.from_pandas(train)
dev_data = Dataset.from_pandas(valid)
test_data = Dataset.from_pandas(test)

encoded_dataset_train = train_data.map(tokenize_function, batched=True)
encoded_dataset_dev = dev_data.map(tokenize_function, batched=True)
encoded_dataset_test = test_data.map(tokenize_function, batched=True)


# tokenized_datasets = train.map(tokenize_function)
encoded_dataset_train = encoded_dataset_train.remove_columns(["text", "__index_level_0__"])
encoded_dataset_train = encoded_dataset_train.rename_column("label", "labels")
encoded_dataset_train.set_format("torch")

encoded_dataset_dev = encoded_dataset_dev.remove_columns(["text", "__index_level_0__"])
encoded_dataset_dev = encoded_dataset_dev.rename_column("label", "labels")
encoded_dataset_dev.set_format("torch")


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
train_dataloader = DataLoader(encoded_dataset_train, shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(encoded_dataset_dev, batch_size=batch_size)

# optimizer and learning rate schedule
optimizer = AdamW(model.parameters(), lr=lr)

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, \
                             num_warmup_steps=0, \
                             num_training_steps=num_training_steps)


from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

metric = load_metric(metric_val)
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

# 80.7 

In [ ]:
# Batch size: 16, 32
# Learning rate (Adam): 5e-5, 3e-5, 2e-5
# Number of epochs: 2, 3, 4
from tqdm.auto import tqdm

batch_size_ls = [16, 32]
lr_ls = [5e-5, 3e-5, 2e-5]
num_epochs_ls = [2,3,4]

for batch_size in batch_size_ls:
  train_dataloader = DataLoader(encoded_dataset_train, shuffle=True, batch_size=batch_size)
  eval_dataloader = DataLoader(encoded_dataset_dev, batch_size=batch_size)

  for lr in lr_ls:

    # optimizer and learning rate schedule
    optimizer = AdamW(model.parameters(), lr=lr)

    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, \
                             num_warmup_steps=0, \
                             num_training_steps=num_training_steps)

    for num_epochs in num_epochs_ls:
      progress_bar = tqdm(range(num_training_steps))
      model.train()
      for epoch in range(num_epochs):
         for batch in train_dataloader:
            batch = {k: v.to(device) for k,v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward() 

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

      metric = load_metric(metric_val)
      model.eval()

      for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
          outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

      print("Results for batch_size = ", batch_size, "and lr = ", lr, "and num_epoch = ", num_epochs)
      print(metric.compute())

# 80.7 

  0%|          | 0/1287 [00:00<?, ?it/s]

Results for batch_size =  16 and lr =  5e-05 and num_epoch =  2
{'f1': 0.7908396946564885}


  0%|          | 0/1287 [00:00<?, ?it/s]

Results for batch_size =  16 and lr =  5e-05 and num_epoch =  3
{'f1': 0.7875000000000001}


  0%|          | 0/1287 [00:00<?, ?it/s]

...

  0%|          | 0/1716 [00:00<?, ?it/s]

Results for batch_size =  16 and lr =  2e-05 and num_epoch =  2
{'f1': 0.790625}


  0%|          | 0/1716 [00:00<?, ?it/s]

Results for batch_size =  16 and lr =  2e-05 and num_epoch =  3
{'f1': 0.7962962962962964}


  0%|          | 0/1716 [00:00<?, ?it/s]